In [3]:
import pandas as pd


## Задание 1

In [33]:
statistics = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SEVERSTAL_task/Data/Статистика за 2018 год.csv', 
                encoding='cp1251')
statistics

,Число бракованных листов,Скорость прокатки,Марка стали
0,3,3.8,A
1,3,3.8,B
2,3,4.8,B
3,4,4.4,B
4,6,5.0,B
...,...,...,...
232,1,4.7,A
233,3,3.4,A
234,2,5.5,A
235,3,5.0,B
